In [1]:
from src.problems.minimum_vertex_cover_vertices import MinimumVertexCoverVertices
from src.choices.logChoice import LogChoice
from src.choices.bigChoice import BigChoice
from src.choicesParameters.fixedParameters import FixedParameters
from src.choicesParameters.metaheuristicParameters import PSOParameters
from src.choicesParameters.transformerParameters import TransformerParametersSVD
from src.problems.utils import generate_random_graph, greedy_dominating_set, smallest_dominating_set
import random

In [2]:
choice = LogChoice()
choiceParameters = FixedParameters()

def fitness(p, problem : MinimumVertexCoverVertices, graph):
    choiceParameters.set(p[0])
    worst_case = None

    for _ in range(10):
        v = problem.run(graph)

        if worst_case is None or worst_case < v:
            worst_case = v
    
    return worst_case

In [3]:
X = []
Y = []

In [4]:
r = []

In [5]:
def get_sequences_from_start(list, top=4):
    return [list[:pos + 1] for pos in range(min(top, len(list)))]

In [8]:
problem = MinimumVertexCoverVertices(choice, choiceParameters)

psoParameters = PSOParameters()
problemFit = MinimumVertexCoverVertices(choice, psoParameters)

for _ in range(300):
    graph = generate_random_graph(random.randint(3, 50))

    score, p = psoParameters.fit(lambda p : fitness(p, problem, graph), choice.apply_constraints, 2, 10, 15)
    #print(score, p)
    
    r.append((graph, score))

    problemFit.run(graph)

    x = get_sequences_from_start(problemFit.embeddings) + [problemFit.embeddings]
    X.extend(x)
    Y.extend([p] * len(x))

In [ ]:
newX = []
newY = []

total = 0
equal = 0
small = 0
big = 0

for (graph, sol), (x, y) in zip(r, zip(X, Y)):
    m = smallest_dominating_set(graph)
    g = greedy_dominating_set(graph)

    if m < g:
        total += 1

        if sol < g:
            small += 1
            newX.append(x)
            newY.append(y)
        elif sol == g:
            equal += 1
        else:
            big += 1

print(total, equal, small, big, len(newX), len(newY))

In [ ]:
transformerParameters = TransformerParametersSVD(2, 2)

In [ ]:
preds, error = transformerParameters.train(newX, newY, 50)
print(error)

In [ ]:
problemTrans = MinimumVertexCoverVertices(choice, transformerParameters)

bad_greedy = 0
best_trans = 0
opt_trans = 0

for _ in range(100):
    graph = generate_random_graph(random.randint(3, 50))

    m = smallest_dominating_set(graph)
    g = greedy_dominating_set(graph)

    if m < g:
        bad_greedy += 1

        t = problemTrans.run(graph)

        if t == m:
            best_trans += 1
            opt_trans += 1
        elif t < g:
            best_trans += 1

print(bad_greedy, best_trans, opt_trans)